In [1]:
'065676'

'065676'

# Imports

In [1]:
import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
from helpers.intraday import *
from helpers.nse_data import *

# Defaults

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In = Investing(check_fresh = True)
Intra = IntraDay()
NSE = NSEData()

Checking Fresh Data.....


# Journal
Make a journal in Google Drive for each trade you take to analyse. It is a not so simple process and need to connect with Google API so if you're comfortable with it, just skip it

In [4]:
private_key = join(expanduser('~'),'Documents','client_secret.json') # Personal for each user. Mine is stored in /home/Documents
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal') # My GoogkeSheet jornam has name "Finance Journal"

JH.total_pl(journal)

--------------------------------------------------------------------------- 

Trailing Stop Loss: 

HDFC bought on: 2021-08-11 00:00:00,

BHARTIARTL bought on: 2021-08-31 00:00:00,

--------------------------------------------------------------------------- 



/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


209.94

# Swing
## All Details
Basic Details about stocks such as Momentum, Ichimoku, 1-2-3 candles pattern etc

In [5]:
budget = 15000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df = df[(df['Overbought'] == False) & (df['Ichi Count'] > 0) & 
        (df['Over 50-SMA'] == True) & (df['Over 100-SMA'] == True) & (df['Over 200-SMA'] == True) &
       (df['Momentum ADX'] > 25) & 
        (df['LTP'] <= budget)]

df.head(2)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Date,Name,LTP,Index,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Overbought,Oversold,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
0,2021-09-16,ACC,2429.70,Nifty 100,True,True,True,True,False,False,31.580264,2,Unknown Red,Bearish Engulfing,Reverse V Pattern
6,2021-09-16,ALKEM,3873.95,Nifty 100,True,True,True,True,False,False,40.899390,2,Unknown Green,Unknown,Unknown


## 44-SMA 
If you find a `Green` candle taking support on 44 days `Simple Moving Average` line and stock is in uptrend, set Risk to Reward ratio as 1:5

In [6]:
budget = 15000
risk = 150
df = In.show_full_stats(budget=budget,risk=risk, diff = risk, nifty='nifty_50')
df = df[(df['Expected Change %'] <= 15) & (df['RSI']<70) & (df['Direction'] != 'Falling Low')]
df

/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:217: UserWarning: Risk should be atleast 161.21 for you to afford INDUSINDBK
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")


,DATE,OPEN,HIGH,LOW,CLOSE,52W H,52W L,SYMBOL,Diff,Direction,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
11,2021-09-16,3798.00,3834.00,3780.05,3813.50,4361.40,2822.45,BAJAJ-AUTO,6.36,Undeterministic,1,55.127559,58.803571,Unknown,Unknown,Green Doji,Nifty 50,4.35,83.54
14,2021-09-16,720.00,729.40,712.85,727.20,734.80,333.75,ICICIBANK,17.42,Rising High,0,63.732911,11.264286,Unknown,Unknown,Unknown Green,Nifty 50,6.72,147.21
0,2021-09-16,780.00,784.55,778.30,781.05,804.35,452.25,SUNPHARMA,20.84,Rising High,0,56.548515,13.778571,Unknown,Unknown,Inverted Hammer,Nifty 50,1.99,148.44
2,2021-09-16,1537.75,1564.30,1536.30,1559.95,1641.00,1025.00,HDFCBANK,24.48,Rising High,0,56.514656,21.803571,V Pattern,Unknown,Unknown Green,Nifty 50,4.14,129.60
12,2021-09-16,792.50,807.45,789.10,802.60,819.00,400.15,AXISBANK,27.55,Rising High,0,63.781310,16.946429,V Pattern,Unknown,Unknown Green,Nifty 50,5.45,132.27
7,2021-09-16,1174.20,1187.00,1168.00,1184.90,1253.65,755.25,SBILIFE,29.08,Undeterministic,0,54.901825,27.607143,Unknown,Unknown,Unknown Green,Nifty 50,4.01,143.11
5,2021-09-16,483.00,488.00,480.45,483.30,488.00,154.40,HINDALCO,44.08,Rising High,0,65.682417,13.792857,Unknown,Unknown,Inverted Hammer,Nifty 50,5.98,146.12
17,2021-09-16,769.00,772.75,760.00,769.65,901.00,312.10,ADANIPORTS,50.22,Undeterministic,0,67.223997,14.707143,Unknown,Unknown,Green Doji,Nifty 50,5.34,144.73
15,2021-09-16,873.50,883.00,871.35,874.00,889.00,458.95,TATACONSUM,58.62,Rising High,0,66.824758,16.453571,Unknown,Unknown,Inverted Hammer,Nifty 50,3.22,142.54
4,2021-09-16,1709.60,1728.50,1709.25,1721.25,1728.50,843.00,LT,76.40,Rising High,0,68.457121,32.707143,Unknown,Unknown,Unknown Green,Nifty 50,4.04,139.70


## Golden Crossover
When 44 days `SMA` Crosses 200 days `SMA` from below and you find a `Green` candle within 15 days taking support on `44` days line set Risk to Reward ratio as 1:5

In [7]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=5):
        print(name)

ESCORTS
ITC


## Oversold / Overbought Stocks
Buy when RSI is less than 30 sell when greater than 70

In [8]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30:
        print(name)

# Intraday

## Narrow Range 7
If recent candle's range is lowest among 6 other days then the future candle might break it's low/high

In [9]:
for name in In.data['nifty_50']:
    if Intra.NR7_strategy(name):
        print(name)

DIVISLAB
HINDALCO
HDFC


## Whole Num Open
When Open == Low / High in Whole number as `xxx.00` within 15 minutes

In [3]:
result = Intra.whole_number_strategy(nifty=50, min_val=101,max_val=4999)
for key in result.keys():
    print(key,":\n")
    for r in sorted(result[key],key=lambda x: int(x[-1].split(' ')[-1])):
        print(r,'\n')

Long :

('BHARTIARTL', 1.91, 'Nifty 50') 

Short :

('JSWSTEEL', -3.37, 'Nifty 50') 

('UPL', -3.02, 'Nifty 50') 



## Indices Based
Buy or Sell stock based on which indices are performing highest in first 5 mins. Buy or sell stocks only for those indices which arevperforming the greatest

In [4]:
NSE.current_indices_status(7)

,index,percentChange,key,variation
8,INDIA VIX,5.84,BROAD MARKET INDICES,0.84
24,NIFTY PSU BANK,-3.23,SECTORAL INDICES,-82.20
26,NIFTY REALTY,-2.61,SECTORAL INDICES,-11.05
22,NIFTY METAL,-2.49,SECTORAL INDICES,-145.00
46,NIFTY COMMODITIES,-1.61,THEMATIC INDICES,-91.85
48,NIFTY CPSE,-1.47,THEMATIC INDICES,-31.90
9,NIFTY MIDCAP 150,-1.43,BROAD MARKET INDICES,-162.25


In [3]:
NSE.open_nse_index('NIFTY PSU BANK')

,symbol,pChange,open,dayHigh,dayLow,lastPrice,Index
13,PNB,-5.02,42.70,42.75,39.35,39.70,Nifty 100
12,BANKBARODA,-4.48,86.55,86.65,81.25,82.05,Nifty 200
11,J&KBANK,-4.17,39.90,40.35,37.30,37.90,Other
10,PSB,-4.03,18.90,19.00,17.50,17.85,Other
9,MAHABANK,-4.01,20.15,20.30,18.65,19.15,Nifty 500
8,BANKINDIA,-3.58,61.00,61.95,57.50,57.90,Nifty 200
7,IOB,-3.52,23.15,23.55,21.30,21.95,Nifty 500
0,NIFTY PSU BANK,-3.18,2573.05,2579.35,2439.70,2461.20,Other
6,UCOBANK,-3.14,14.45,14.55,13.75,13.90,Nifty 500
5,UNIONBANK,-3.13,37.20,37.45,35.35,35.55,Nifty 200


# Get Particulars

In [11]:
In.get_particulars('DABUR',10000,100,delta=0.005)#, entry = 620.1 , stop_loss=587.9)  # 620.1, 587.9

{'Buying Price': 648.88,
 'Stop-Loss %': 2.43,
 'Target %': 4.86,
 'Quantity': 6.0,
 'Stop-Loss Price': 633.1184999999999,
 'Trigger Price': 680.38,
 'Risk Per Share': 15.76,
 'Profit Per Share': 31.5,
 'Max loss on this config': 94.56,
 'Max Gain on this config': 189.02,
 'Price To Profit Ratio': 20.59,
 'Index': 'Nifty 100'}